In [ ]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from shapely import wkb
import geopandas
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)[0]

In [ ]:
# CONECTAR A AMIGOCLOUD
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [ ]:
def generar_reporte(codigo_canhero):
    # EXTRAER DATOS DE CREDITO
    query_credito = {'query': 'select * from dataset_282969 where cast(SPLIT_PART(canhero, \'/\', 1) as integer) = {cod_ca}'.format(cod_ca=codigo_canhero)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/31360/sql', query_credito)
    credito_data = credito_select['data'][0]
    credito_data['fecha_informe'] = datetime.strptime(credito_data['fecha_2da_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    credito_data['fecha_2da_insp'] = datetime.strptime(credito_data['fecha_2da_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())

    # EXTRAER DATOS DE PROPIEDADES
    # query_propiedades = {'query': 'select * from dataset_282968 where cre_ref_prop = \'{cre_ref_prop}\''.format(cre_ref_prop=credito.amigo_id)}
    query_propiedades = {'query': 'select prop.propiedad, prop.amigo_id from dataset_282968 prop inner join dataset_282975 lotes on lotes.prop_ref_lote=prop.amigo_id where cre_ref_prop=\'{cre_ref_prop}\' and (lotes.estado_de_siembra=\'CAÑA NACIDA\' or lotes.estado_de_siembra=\'CAÑA NO NACIDA\') group by prop.propiedad, prop.amigo_id'.format(cre_ref_prop=credito.amigo_id)}
    propiedades_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/31360/sql', query_propiedades)    
    
    propiedades_data = propiedades_select['data']

    # EXTRAER DATOS DE LOTES
    propiedades = []
    for prop in propiedades_data:
        prop['lotes'] = []
        query_lotes = {'query': 'select * from dataset_282975 where prop_ref_lote=\'{prop_ref_lote}\' and (estado_de_siembra=\'CAÑA NACIDA\' or estado_de_siembra=\'CAÑA NO NACIDA\')'.format(prop_ref_lote=prop['amigo_id'])}
        lotes_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/31360/sql', query_lotes)
        for lote_select in lotes_select['data']:
            lote = collections.namedtuple("lote", lote_select.keys())(*lote_select.values())
            prop['lotes'].append(lote)
        propiedades.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))

    # EXTRAER FOTOS
    fotos = []
    for propiedad in propiedades:
        for lote in propiedad.lotes:
            print(lote.lote, lote.amigo_id)
            query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_41588 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=lote.amigo_id)}
            fotos_filename = amigocloud.get('https://app.amigocloud.com/api/v1/projects/31360/sql', query_fotos)
            if fotos_filename['data'] != []:
                dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
                fotos.append(dic)

    # GENERERAR PLANOS
    i = 0
    lista_planos = []
    for propiedad in propiedades:
        i = i + 1
        lista_planos.append(generar_plano(propiedad, i))


    # GENERAR REPORTE
    doc = DocxTemplate("templates/template_word_2da.docx")
    #generar lista de InlineImage de planos 
    lista_InlineImage = []
    for plano in lista_planos:
        lista_InlineImage.append(docxtpl.InlineImage(doc, image_descriptor=plano, width=Mm(150)))
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        file = open('fotos/' + foto['file_name'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['file_name'], width=Mm(120)), 'lote': foto['lote']})
    firma_respon = None
    if credito.respon_1ra_insp == 'Rogelio Acuña Rodríguez':
        firma_respon = docxtpl.InlineImage(doc, image_descriptor='templates/firma_rogelio.png', width=Mm(60))
    else:
        firma_respon = docxtpl.InlineImage(doc, image_descriptor='templates/firma_jaldin.png', width=Mm(60))
    context = {'credito':credito, 'propiedades':propiedades, 'planos':lista_InlineImage, 'fotos':lista_fotos_inline, 'firma':firma_respon}
    doc.render(context)
    file_name = credito.canhero.replace(' / ', '_')
    doc.save('informes/_' + file_name + '.docx')

In [ ]:
def generar_plano(propiedad, i):
    lotes_lista = []
    for lote in propiedad.lotes:
        lotes_lista.append(lote._asdict())
    df = pd.DataFrame(lotes_lista)
    df['geometria'] = df['geometria'].apply(convertir_wkb)

    #Convertir a GeoDataFrame
    data = geopandas.GeoDataFrame(df, geometry='geometria')
    #data
    #ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)

    data['coords'] = data['geometria'].apply(lambda x: x.representative_point().coords[:])
    data['coords'] = [coords[0] for coords in data['coords']]

    data.crs = "EPSG:4326"
    data = data.to_crs(epsg=3857)

    
    fig = plt.figure(i, figsize=(20,20))
    #fig = plt.figure(1)
    ax = None
    ax = fig.add_subplot()

    #data.set_crs(epsg=4326, inplace=True)

    data.apply(lambda x: ax.annotate(text=x.lote + ' \n' + str(x.superficie) + ' ha', xy=x.geometria.centroid.coords[0], ha='center', va='center', color='black', fontsize=12, weight=1000, bbox=dict(facecolor=(1,1,1,0.3), edgecolor='none', pad=0)), axis=1);

    #data.boundary.plot(ax=ax, color=(0,0,0,0), linewidth=0)

    #data = data.to_crs(epsg=3857)

    minx, miny, maxx, maxy = data.total_bounds
    ax.set_xlim(minx - 500, maxx + 500)
    ax.set_ylim(miny - 400, maxy + 400)

    data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2, figsize=(20,20))
    #data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2)
    
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
    ax.set_axis_off()
    ax.set_title(str(propiedad.propiedad), fontsize=20)
    ruta = 'planos/' + str(propiedad.amigo_id) + '.jpeg'
    fig.savefig(ruta, dpi = 300, bbox_inches='tight')
    plt.clf()
    return ruta

In [ ]:
semana = 'S23'
query = {'query': 'select split_part(canhero, \' / \', 1) cod, split_part(canhero, \' / \', 2) nom from dataset_282969 where semana_2da = \'{sem}\''.format(sem=semana)}
project = 'https://app.amigocloud.com/api/v1/projects/31360/sql'
select = amigocloud.get(project, query)
data = select['data']

In [ ]:
data

In [ ]:
# lista = [430, 40270, 13132, 874, 41966, 11798, 41544, 14993, 644, 1068, 13763, 2220, 1858, 2719]
lista = [i['cod'] for i in data]

In [ ]:
lista

In [ ]:
for i in lista:
    generar_reporte(i)
    print('======================', i)